<a href="https://colab.research.google.com/github/PabloAndree/Python_PA/blob/main/Primer_entregable_ML_Grupo_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.utils import shuffle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from scipy.stats import zscore
import numpy as np
import seaborn as sns
import math
import copy
from scipy import stats
import re
from statistics import mode
from scipy import stats as s
from pathlib import Path
from sklearn.cluster import KMeans
import zipfile
import warnings
warnings.filterwarnings("ignore")

In [6]:
root_tph = pd.read_csv('/content/drive/My Drive/UTEC-GRUPO 3/fuentes de datos/tph.csv')
root_blm = pd.read_csv('/content/drive/My Drive/UTEC-GRUPO 3/fuentes de datos/blocks_model.csv')
root_tds = pd.read_csv('/content/drive/My Drive/UTEC-GRUPO 3/fuentes de datos/control_sense.csv')
root_pf = pd.read_csv('/content/drive/My Drive/UTEC-GRUPO 3/fuentes de datos/pfinos_l1.csv')
root_dsh = pd.read_csv('/content/drive/My Drive/UTEC-GRUPO 3/fuentes de datos/geologia.csv')
root_drill = pd.read_csv('/content/drive/My Drive/UTEC-GRUPO 3/fuentes de datos/perforacion.csv')
root_epiroc = pd.read_csv('/content/drive/My Drive/UTEC-GRUPO 3/fuentes de datos/hole_information_1.csv')
root_asp = pd.read_csv('/content/drive/My Drive/UTEC-GRUPO 3/fuentes de datos/assaysamples.csv')

Paso 2: Renombramos data frame

In [8]:
root_datasets = [root_tph,root_blm,root_tds,root_pf,root_dsh,root_drill,root_epiroc,root_asp]

In [13]:
list_datasets = []
for root in root_datasets:
  list_datasets.append(root)
df_tph,df_blm,df_tds,df_pf,df_dsh,df_drill,df_epiroc,df_asp = list_datasets[0],list_datasets[1],list_datasets[2],list_datasets[3],list_datasets[4],list_datasets[5],list_datasets[6],list_datasets[7]

In [24]:
#Separa fechas en time y redondear los decimales en value
df_tph.head()

,tag,time,value
0,3213WIC0322A_PID_PV_Out_Value,2023-06-03 00:00:00.0000000,2314.202460
1,3213WIC0322A_PID_PV_Out_Value,2023-06-03 00:05:00.0000000,2308.289051
2,3213WIC0322A_PID_PV_Out_Value,2023-06-03 00:10:00.0000000,2294.576673
3,3213WIC0322A_PID_PV_Out_Value,2023-06-03 00:15:00.0000000,2300.387351
4,3213WIC0322A_PID_PV_Out_Value,2023-06-03 00:20:00.0000000,2289.532840


In [28]:
df_blm.head()

,id,east,north,elev,sg,material,cupct,cucn,sec,cuss,...,espac,dtime,expkg,hexpl,napct,kpct,mgpct,alpct,nsr2,matt0
0,1,200665.0,8399815.0,4252.5,2.29,12,0.07599,0.00974,12.82,0.01250,...,-1.0,1.67,-1.0,-1.0,2.56850,2.41539,1.11993,7.97270,0.000,8.0
1,2,200665.0,8399825.0,4252.5,2.30,4,0.07575,0.01146,15.13,0.01523,...,-1.0,1.80,-1.0,-1.0,2.70585,1.71900,1.47856,8.40256,0.000,7.0
2,3,200665.0,8399835.0,4252.5,2.49,4,0.09874,0.00320,3.24,0.01436,...,-1.0,1.61,-1.0,-1.0,1.39200,1.68160,1.99001,8.89063,0.000,8.0
3,4,200665.0,8399845.0,4252.5,2.47,7,0.12562,0.00550,4.38,0.01417,...,-1.0,2.64,-1.0,-1.0,2.60153,1.47882,1.55715,8.82272,6.577,8.0
4,5,200665.0,8399855.0,4252.5,2.42,7,0.13295,0.00915,6.88,0.01132,...,-1.0,3.40,-1.0,-1.0,3.65968,1.32217,1.39043,8.18794,6.934,8.0


In [29]:
#Separar las fechas en tiempo actualizacion y quitar los valores NaN
df_tds.head()

,control_sense_id,tiempo_actualizacion,coordenada_x_carga,coordenada_y_carga,coordenada_z_carga,coordenada_x_descarga,coordenada_y_descarga,coordenada_z_descarga,equipo_carguio,equipo_acarreo,...,ley_viaje_alpct,ley_viaje_capct,ley_viaje_nsr2,ley_viaje_matt0,viaje_lito,viaje_alter,viaje_pvbh,x_centroide,y_centroide,z_centroide
0,2790757,2023-12-20 11:40:35.4690000,"20164962,20164962,20165147,20165147","839782227,839782227,839782579,839782579","422323,422323,422401,422401",20008021.0,839875396.0,414590.0,CF002,CM28,...,0.000000,3.241389,0.000000,0.000000,0.0,0.0,0.0,"20165250,20164750,20165250,20164750","839782750,839782250,839782750,839782250","422750,422750,422750,422750"
1,2695146,2023-08-30 12:21:57.5140000,"20455237,20455994,20455834,20454950,20455753","839688496,839688175,839688331,839688580,839688283","416221,416209,416060,415920,415998",20008081.0,839874298.0,417015.0,PH003,CM17,...,4.334636,13.100973,33.219032,9.000000,3.0,8.0,12321.0,NaN,NaN,NaN
2,2564735,2023-04-16 16:40:01.5700000,"20457165,20457128,20457353,20457475,20457218,2...","839674242,839674035,839674026,839673873,839674...","420596,420624,420580,420464,420739,420461,420643",20231855.0,839768693.0,424648.0,PH003,CM15,...,4.544465,11.196750,16.616951,9.000000,5.0,8.0,12033.0,NaN,NaN,NaN
3,2631160,2023-06-27 04:08:15.7850000,"20075535,20075733,20076159,20076396,20076378,2...","839986963,839987014,839987156,839987265,839987...","419331,419242,419046,418941,418977,419021,418964",20008030.0,839874147.0,416509.0,PH001,CM10,...,5.305685,0.625604,5.181423,4.490454,1.0,3.0,42146.0,NaN,NaN,NaN
4,2790464,2023-12-20 03:54:58.3590000,"20163939,20163939,20163939,20163939,20165092","839784388,839784388,839784388,839784388,839783851","422993,422993,422993,422993,422418",20008025.0,839874517.0,416644.0,CF002,CM07,...,0.000000,3.241389,0.000000,0.000000,0.0,0.0,0.0,"20163750,20165250,20163750,20163750,20163750","839784250,839783750,839784250,839784250,839784250","422750,422750,422750,422750,422750"


In [30]:
#Separa fechas en time y redondear los decimales en value
df_pf.head()

,tag,time,value
0,Faja_CV001_C16_F10_C16_PV_Out_Value,2023-06-03 00:00:00.0000000,0.028080
1,Faja_CV001_C16_F10_C16_PV_Out_Value,2023-06-03 00:05:00.0000000,0.026350
2,Faja_CV001_C16_F10_C16_PV_Out_Value,2023-06-03 00:10:00.0000000,0.029083
3,Faja_CV001_C16_F10_C16_PV_Out_Value,2023-06-03 00:15:00.0000000,0.024509
4,Faja_CV001_C16_F10_C16_PV_Out_Value,2023-06-03 00:20:00.0000000,0.034682


In [31]:
#Quitar los valores NaN
df_dsh.head()

,DataSet,Hole_ID,Depth_From,Depth_To,Interval_Length,Priority,Lith_Plot,Lith1_Code,Lith1_Texture,Lith1_Structure,...,MinZ_pct,MinZ2_pct,EXP_Kg,burden,Espac,altura,FP,malla,T_malla,Explosivo
0,CN,08-95-0001,0.0,10.0,10.0,0,NaN,11,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CN,08-95-0002,0.0,10.0,10.0,0,NaN,11,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CN,08-95-0003,0.0,10.0,10.0,0,NaN,11,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CN,08-95-0004,0.0,10.0,10.0,0,NaN,11,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CN,08-95-0005,0.0,10.0,10.0,0,NaN,11,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
#Separa las fechas
df_drill.head()

,controlsenseid,bhid,fecha,fechamina,hora,turno,estep,nortep,zp,ested,...,proveedorbroca,deckbushing,proveedordeckbushing,tiemperforando,tiemtrasladando,tiemacomodando,tiemcreac,tiemdemontando,tiemupdate,tiemelimin
0,287090,20-01-8227,2022-10-17 07:11:01.000,2022-10-17,07:11:01,A,204540.09,8396956.25,4215.42,204540.14,...,OTRO,sandvik s-4600402,OTRO,2022-10-17 07:48:51.793,2022-10-17 07:11:01.260,2022-10-17 07:48:57.823,2022-10-17 07:50:38.747,2022-10-17 07:11:09.033,2022-10-17 08:30:01.197,NaN
1,287096,20-01-8356,2022-10-17 07:59:13.000,2022-10-17,07:59:13,A,204601.18,8396957.25,4215.71,204601.26,...,OTRO,SANDVIK S-4754303,OTRO,2022-10-17 08:25:21.300,2022-10-17 07:59:13.253,2022-10-17 08:25:24.667,2022-10-17 08:27:56.500,2022-10-17 07:59:22.867,2022-10-17 08:30:01.197,NaN
2,287097,27-04-9233,2022-10-17 08:03:58.000,2022-10-17,08:03:58,A,201482.13,8399339.73,4109.63,201482.12,...,OTRO,sandvik. s_4754301,OTRO,2022-10-17 08:14:53.320,2022-10-17 08:03:58.587,2022-10-17 06:16:16.927,2022-10-17 11:10:09.640,2022-10-17 08:04:31.670,2022-10-17 08:15:18.047,NaN
3,287098,20-01-8295,2022-10-17 08:27:59.000,2022-10-17,08:27:59,A,204606.63,8396956.58,4215.50,204606.69,...,OTRO,SANDVIK S-4754303,OTRO,2022-10-17 08:47:39.640,2022-10-17 08:27:59.973,2022-10-17 08:47:44.453,2022-10-17 08:50:22.497,2022-10-17 08:28:11.187,2022-10-17 08:47:52.910,NaN
4,287095,20-01-8228,2022-10-17 07:51:50.000,2022-10-17,07:51:50,A,204545.46,8396955.32,4215.52,204545.54,...,OTRO,sandvik s-4600402,OTRO,2022-10-17 09:19:14.787,2022-10-17 07:51:50.903,2022-10-17 09:19:51.847,2022-10-17 09:21:17.680,2022-10-17 07:51:58.467,2022-10-17 09:19:23.030,NaN


In [34]:
#Separa las fechas
df_epiroc.head()


,MwdHoleId,MwdSampleId,DrillPlanId,DrillMethodId,HoleId,HoleName,DrillMethodName,PlanId,RigSerialNumber,RigName,EquipmentType,Depth,Time,PenetrationRate,FlushAirPressure,FeederPressure,RotationPressure,RotationTorque
0,C7572AEB-6124-479B-8F1C-B02A001C5721,CFA93F5F-8809-43E6-9C61-B02A001C5721,BB417262-E516-4850-994B-ACC300862707,6DF319C1-E552-410F-8116-CA2CB1D29AAF,-1.0,4327855.0,Rotary,AutoPlan,8999482100,DR002,ADS PV271,5.537,2023-06-23 01:27:02,0.988,0.0,0.0,0.0,5.228
1,0F0B8C4D-51BA-4677-8C1C-B02A003EAE2E,40A7D52E-AA27-4E6E-90B2-B02A003EAE2F,BB417262-E516-4850-994B-ACC300862707,6DF319C1-E552-410F-8116-CA2CB1D29AAF,-1.0,4327855.0,Rotary,AutoPlan,8999482100,DR002,ADS PV271,12.305,2023-06-23 03:38:26,0.995,0.0,0.0,0.0,6.439
2,FA69A6A4-CC1F-4153-9D8B-B02A0000948E,6D0D3CDA-5260-4AC6-B247-B02A0000948F,BB417262-E516-4850-994B-ACC300862707,6DF319C1-E552-410F-8116-CA2CB1D29AAF,-1.0,4327855.0,Rotary,AutoPlan,8999482100,DR002,ADS PV271,12.827,2023-06-22 23:56:50,1.979,0.0,0.0,0.0,6.665
3,421E6CD2-39B1-41F6-8ABB-B02A007E3850,5F4A651B-648F-4775-AD2B-B02A007E3850,BB417262-E516-4850-994B-ACC300862707,6DF319C1-E552-410F-8116-CA2CB1D29AAF,-1.0,4324344.0,Rotary,AutoPlan,8999482100,DR002,ADS PV271,7.038,2023-06-23 07:26:58,1.440,0.0,0.0,0.0,7.434
4,382A8E96-AFE1-4D9B-8931-B02A00903E09,1C497116-EE9F-4740-AF62-B02A00903E09,BB417262-E516-4850-994B-ACC300862707,6DF319C1-E552-410F-8116-CA2CB1D29AAF,-1.0,1215213.0,Rotary,AutoPlan,8999483100,DR003,ADS PV271,13.934,2023-06-23 08:38:42,5.758,0.0,0.0,0.0,10.120


In [35]:
#Separa las fechas de data sampled
df_asp.head()

,DataSet,SampleID,Hole_ID,Au_ppm,Pb_ppm,Cu_ppm,Date_Sampled,Au_ppb,Ag_ppm,Al_pct,...,F_ppm,Eu_ppb,Er_ppb,Dy_ppb,Ce_ppb,Br_ppm,B_ppm,NAT_East,NAT_North,NAT_RL
0,CO,109,05-01-0134,5.0,180.0,5.0,2014-03-06 00:00:00.000,5000.0,20.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201829.844,8399711.272,4440.0
1,CO,119,05-01-0144,5.0,50.0,820.0,2014-03-07 00:00:00.000,5000.0,20.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201856.829,8399673.358,4440.0
2,CO,149,05-01-0173,5.0,70.0,5.0,2014-03-07 00:00:00.000,5000.0,20.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201837.021,8399640.396,4440.0
3,CO,159,05-01-0182,5.0,5.0,5.0,2014-03-07 00:00:00.000,5000.0,20.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201813.193,8399640.898,4440.0
4,CO,169,06-01-0055,5.0,400.0,300.0,2014-03-28 00:00:00.000,5000.0,20.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201976.954,8399764.689,4425.0


Paso 3: Filtrando datos

In [ ]:
#DETERMINAR LAS VARIABLES A TRABAJAR EN CADA DF

Paso 4: Limpieza de datos

In [ ]:
#EMPLEAR FUNCIONES DE LIMPIEZA Y UNIFORMIDAD DE DATOS

Paso 5: Consolidación de datos en un solo DF

In [ ]:
#¿CON QUE VARIABLES REALIZAREMOS LA CONSOLIDACIÓN?